# PAA -  Greed 



### PT 1. Estrutura inicial
O primeiro passo é definir a estrutura da nossa heap, que será utilizada para guardar cada um dos nosso caracteres em conjunto com sua frequência de ocorrência e nós a esquerda e a direita na árvore. 

In [5]:
import heapq
import os
class HeapNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

    def __eq__(self, other):
        if(other == None):
            return False
        if(not isinstance(other, HeapNode)):
            return False
        return self.freq == other.freq

Agora podemos pedir ao usuário o caminho do arquivo a ser comprimido. Em seguida preenchemos o nosso heap com esses dados

In [6]:
while True:
    original_path = input("Digite o caminho do arquivo:")
    if os.path.isfile(original_path):
        break
    print("Arquivo não encontrado, tente novamente.")

original_filename, original_file_extension = os.path.splitext(original_path)
with open(original_path, 'r+') as file:
    text = file.read()
    text = text.rstrip()

chars_frequency = {}
for character in text:
    if not character in chars_frequency:
        chars_frequency[character] = 0
    chars_frequency[character] += 1

print(chars_frequency)

heap = []

for key in chars_frequency:
    node = HeapNode(key, chars_frequency[key])
    heapq.heappush(heap, node)

Digite o caminho do arquivo:asd
Arquivo não encontrado, tente novamente.
Digite o caminho do arquivo:asd.txt
Arquivo não encontrado, tente novamente.
Digite o caminho do arquivo:test.txt
{'A': 1, ' ': 9, 'm': 3, 'i': 5, 'r': 4, 'a': 15, 'l': 7, 'g': 3, 'u': 2, 'c': 1, 'o': 5, 's': 6, '\n': 5, 't': 5, 'e': 6, 'd': 7, 'n': 2, 'j': 2, 'b': 1, 'f': 1, 'v': 1}


Agora construimos nossa árvore com os caracteres, de forma que os nós mais próximos da raiz são aqueles que representam os caracteres com maior frequência. Isso fica especialmente fácil por termos utilizado uma heap, já guardando os nós em ordem de frequência. 

In [8]:
while(len(heap)>1):
    node1 = heapq.heappop(heap)
    node2 = heapq.heappop(heap)

    merged = HeapNode(None, node1.freq + node2.freq)
    merged.left = node1
    merged.right = node2

    heapq.heappush(heap, merged)

Estando com a árvore montada, podemos construir os códigos para cada caracter.

In [9]:

reverse_mapping = {}
codes = {}

def build_huffman_code(root, current_code):
    if(root == None):
        return

    if(root.char != None):
        codes[root.char] = current_code
        reverse_mapping[current_code] = root.char
        return

    build_huffman_code(root.left, current_code + "0")
    build_huffman_code(root.right, current_code + "1")
    
    
build_huffman_code(heapq.heappop(heap), "")
print(codes)

{'i': '0000', 'r': '0001', 'm': '0010', 'b': '00110', 'l': '00111', 'e': '010', 's': '0110', '.': '011100', 'L': '01110100', 'j': '01110101', 'z': '0111011', 'd': '01111', 'n': '10000', 'c': '10001', 'M': '1001000', 'h': '1001001', 'x': '10010100', 'T': '100101010', 'S': '100101011', 'g': '1001011', '\n': '10011', 't': '101000', 'v': '101001', 'u': '10101', 'o': '1011', 'a': '1100', 'Q': '1101000', 'f': '1101001', 'V': '1101010', 'ê': '1101011', ',': '110110000', 'q': '110110001', 'ó': '11011001', 'p': '1101101', 'ã': '11011100', 'É': '110111010', 'á': '1101110110', 'N': '11011101110', 'A': '11011101111', '?': '11011110', 'E': '11011111', ' ': '111'}


In [10]:
encoded_text = ""
for character in text:
    encoded_text += codes[character]
print(encoded_text)

1101000101011100100000111110111110101010111110100001011110100101001110101101110011110111110110110110101000011011111101000010101011110011001000000111010110111001111011101110110111001011111011000001000010100010111111010010100001100101110111000010010011100100111101110111111011010101000011000110111011110100110010011101011011100111001000000010000100100111001110011010111000111001110101000010001101101101010001100100111101111100101111010001010111001110011010111000111001111101100011010101011110100000010100010010100111001000010101010110111101100111100100110110110111010101011111101001010100011001001101110011100100011000110111101101101111010000101010101101110100110101000110111001011111110000110010000110100010110110100111001010101010101111101111100110010001011101100101111000011011100101111101111010101011111000101000011010001011100111101111110101111110010001100100101000001111101100011010101011110000110110010110111100011001001010100101111000010101101101111010001101110010111111101101010000110100010111

In [11]:
extra_padding = 8 - len(encoded_text) % 8
for i in range(extra_padding):
    encoded_text += "0"

padded_info = "{0:08b}".format(extra_padding)
encoded_text = padded_info + encoded_text

In [12]:
compressed_path=f'{original_filename}_compressed.bin'
with open(compressed_path, 'wb') as output:

    btarray = bytearray()
    for i in range(0, len(encoded_text), 8):
        byte = encoded_text[i:i+8]
        btarray.append(int(byte, 2))        
        
    output.write(bytes(btarray))

In [13]:
original_file_size = os.path.getsize(original_path)
compressed_file_size = os.path.getsize(compressed_path)
size_reduction = format(round((((original_file_size-compressed_file_size)/original_file_size)*100), 0))
print(f'Nome do arquivo comprimido: {compressed_path}')
print(f'Tamanho do arquivo original: {original_file_size} bytes')
print(f'Tamanho do arquivo comprimido: {compressed_file_size} bytes')
print(f'O algoritmo conseguiu reduzir o tamanho do arquivo para {size_reduction}% do seu tamanho original')

Nome do arquivo comprimido: lorem_compressed.bin
Tamanho do arquivo original: 1427 bytes
Tamanho do arquivo comprimido: 780 bytes
O algoritmo conseguiu reduzir o tamanho do arquivo para 45.0% do seu tamanho original


In [14]:
compressed_path = input("Digite o caminho do arquivo a ser descomprimido: ")
compressed_filename, compressed_file_extension = os.path.splitext(compressed_path)

with open(compressed_path, 'rb') as file:
    bits_string = ""

    byte = file.read(1)
    while(len(byte) > 0):
        byte = ord(byte)
        bits = bin(byte)[2:].rjust(8, '0')
        bits_string += bits
        byte = file.read(1)

Digite o caminho do arquivo a ser descomprimido: lorem_compressed.bin


In [15]:
padded_info = bits_string[:8]
extra_padding = int(padded_info, 2)

bits_string = bits_string[8:] 
encoded_text = bits_string[:-1*extra_padding]

In [16]:
current_code = ""
decompressed_text = ""

for bit in encoded_text:
    current_code += bit
    if(current_code in reverse_mapping):
        character = reverse_mapping[current_code]
        decompressed_text += character
        current_code = ""
print(decompressed_text)

Quando eu te vejo
Espero teu beijo
Não sinto vergonha
Apenas desejo
Minha boca encosta
Em tua boca que treme
Meus olhos eu fecho
Mas os teus estão abertos
Tudo bem se não deu certo
Eu achei que nós chegamos tão perto
Mas agora, com certeza eu enxergo
Que no fim eu amei por nós dois
Esse foi um beijo de despedida
Que se dá uma vez só na vida
Que explica tudo sem brigas
E clareia o mais escuro dos dias
Tudo bem se não deu certo
Eu achei que nós chegamos tão perto
Mas agora, com certeza eu enxergo
Que no fim eu amei por nós dois
Mas você lembra? Você vai lembrar de mim
Que o nosso amor valeu a pena
Lembra? É o nosso final feliz
Você vai lembrar... Vai lembrar... Sim
Você vai lembrar de mim
Esse foi um beijo de despedida
Que se dá uma vez só na vida
Que explica tudo sem brigas
E clareia o mais escuro dos dias
Tudo bem se não deu certo
Eu achei que nós chegamos tão perto
Mas agora, com certeza eu enxergo
Que no fim eu amei por nós dois
Mas você lembra? Você vai lembrar de mim
Que o nosso am

In [17]:
decompressed_file_path=f'{compressed_filename}_decompressed.txt'
with open(decompressed_file_path, 'w') as output:
    output.write(decompressed_text)
print(f'O arquivo descompactado foi salvo como {decompressed_file_path}')

O arquivo descompactado foi salvo como lorem_compressed_decompressed.txt


In [18]:
import filecmp
filecmp.cmp(original_path, decompressed_file_path)

True